# Fusion-Chef Exercise

This notebook implements a Fusion-Chef bot that:
1. Combines two dishes into a creative fusion dish
2. Suggests matching cocktails
3. Stores successful recipes in memory

In [ ]:
import os
import sys

notebook_dir = os.path.abspath("")
parent_dir = os.path.dirname(notebook_dir)
grandparent_dir = os.path.dirname(parent_dir)

sys.path.append(grandparent_dir)

### 1. Initialize Kernel and Services 

In [ ]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureTextEmbedding
from semantic_kernel.memory.semantic_text_memory import SemanticTextMemory
from semantic_kernel.memory.volatile_memory_store import VolatileMemoryStore
from semantic_kernel.core_plugins.text_memory_plugin import TextMemoryPlugin

kernel = Kernel()

service_id = "chat-gpt"
chat_service = AzureChatCompletion(
    service_id=service_id,
)

embedding_service_id = "embeddings"
kernel.add_service(AzureTextEmbedding(service_id=embedding_service_id))

kernel.add_service(chat_service)

### 2. Setup Memory 

In [ ]:
memory_store = VolatileMemoryStore()
memory = SemanticTextMemory(storage=memory_store, embeddings_generator=kernel.get_service(embedding_service_id))
kernel.add_plugin(TextMemoryPlugin(memory), "TextMemoryPlugin")

### 3. Define Prompts (experiment with these!)

In [ ]:
# Prompt for creating fusion dishes
fusion_prompt = """
Dish 1: {{$dish1}}
Dish 2: {{$dish2}}

Create a unique fusion dish that combines elements from both dishes.
Be creative but make sure the combination makes culinary sense.

Provide the response in this format:
🍽️ Fusion Name: [creative name for the dish]
📋 Description: [brief description]
🥘 Key Ingredients: [main ingredients]
👨‍🍳 Preparation: [key steps]
"""

# Prompt for suggesting cocktails
cocktail_prompt = """
Fusion Dish: {{$fusion_dish}}

Suggest a creative cocktail that would pair well with this fusion dish.
Consider the flavors and cultural elements of the dish.

Provide the response in this format:
🍸 Cocktail Name: [creative name]
🌟 Style: [type of cocktail]
🧉 Ingredients: [list of ingredients]
✨ Why it pairs well: [brief explanation]
"""

# Create semantic functions
fusion_function = kernel.add_function(
    prompt=fusion_prompt,
    function_name="create_fusion",
    plugin_name="FusionChef",
)

cocktail_function = kernel.add_function(
    prompt=cocktail_prompt,
    function_name="suggest_cocktail",
    plugin_name="FusionChef",
)

### 4. Example Usage

In [ ]:
# Create some fusion dishes
dish_pairs = [
    ("Sushi", "Tacos"),
    ("Pizza", "Pad Thai"),
    ("Hamburger", "Dim Sum")
]

for dish1, dish2 in dish_pairs:
    # Create fusion dish
    fusion = await kernel.invoke(
        fusion_function,
        dish1=dish1,
        dish2=dish2
    )
    print(f"\nFusing {dish1} with {dish2}:\n{fusion}")
    
    # Get cocktail pairing
    cocktail = await kernel.invoke(
        cocktail_function,
        fusion_dish=str(fusion)
    )
    print(f"\nSuggested Pairing:\n{cocktail}")
    
    # Store successful recipe in memory
    await memory.save_information(
        collection="recipes",
        id=f"fusion_{dish1}_{dish2}".lower().replace(" ", "_"),
        text=f"Fusion Dish:\n{fusion}\n\nCocktail Pairing:\n{cocktail}"
    )

### 5. Search Saved Recipes

In [ ]:
# Search for recipes by ingredient or style
search_terms = ["spicy", "seafood", "Asian fusion"]

for term in search_terms:
    print(f"\nSearching recipes related to '{term}':\n")
    results = await memory.search("recipes", term)
    for result in results:
        print(f"Match (relevance: {result.relevance}):\n{result.text}\n")

### 6. Additional exercises

Try these exercises:
1. Modify the fusion_prompt to include nutritional information
2. Add a new prompt to create vegetarian versions of fusion dishes
3. Create a function to rate the creativity of fusion combinations
4. Add a prompt to suggest wine pairings instead of cocktails
5. Implement a way to filter recipes by dietary restrictions